<a href="https://colab.research.google.com/github/TheKVT/Artwork_Extractor/blob/master/Artwork_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown <h3> 👈  Run This Cell to Mount Gdrive</h3> 
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

#@title This Cell to Extract Album Art from FLAC Audio
Flac_Path = "" #@param {type:"string"}
Save_Path = "" #@param {type:"string"}
 
import IPython
import ipywidgets as widgets
 
from IPython.display import HTML, clear_output
print("Downloading...") 
get_ipython().system_raw("rm -rf /content/sample_data/ && sudo apt update && sudo apt install software-properties-common")
get_ipython().system_raw("pip3 -q install mutagen")
 
# %cd $Save_Path
clear_output()
 
from mutagen.flac import FLAC, Picture
song = Flac_Path
var = FLAC(song)
pics = var.pictures
print (pics)
for p in pics:
    if p.type == 3: #front cover
        display(HTML("<center><h2 style=\"font-family:Merriweather;color:#00b24c;\">Album Art Successfully Exported!</h2><br></center>")) 
        with open("cover.jpg", "wb") as f:
            f.write(p.data)

In [ ]:

##################################################
#
# Code Author: jamescarter2001
#
# Remastered by TheKVT
#
##################################################

# ============================= FORM ============================= #
#@title This Cell can extract  Digital Album Artwork of [Apple Music](https://music.apple.com/) and [Bugs!](https://music.bugs.co.kr/)
Mode = "Apple" #@param ["Apple", "Bugs"]
# @markdown > Select mode (Apple Music / Bugs)
# @markdown ---
url = "" #@param {type:"string"}
Picture_Resoultion = "3000" #@param ["1000", "1500", "2000", "2500", "3000"]
country = "us" #@param {type:"string"}
# @markdown > `country` only needed for `Apple`
output_path = "" #@param {type:"string"}

# ================================================================ #

import requests
import json
import os
import pycountry
import wget
from urllib.request import urlopen
from bs4 import BeautifulSoup
from IPython.display import clear_output
os.makedirs(output_path, exist_ok=True)

!pip install pycountry wget bs4
clear_output()

############## Apple Music ############
if Mode == "Apple": 
   res = Picture_Resoultion

   region = pycountry.countries.get(alpha_2=country)

   language = "en_us"  # ja_jp

   link = url.split('/')[-1].split('?')[0]

   itunes_endpoint = f'https://itunes.apple.com/lookup?id={link}&country={region.alpha_2}&lang={language}'

   response = requests.get(itunes_endpoint)
   if response.status_code == 200: 
    file_name = response.json()["results"][0]["collectionName"]
    artwork_link = response.json()["results"][0]["artworkUrl100"]
    artwork_link = artwork_link.replace("100x100bb.jpg", f"{res}x{res}bb.jpg")
    url = artwork_link
    path = output_path
    wget.download(url, f'{path}/{file_name}-cover.jpg')
    print("Downloaded " + file_name + " Album ArtWork Sucessfully..!!")
    
## if you get "index error" means 'item not avilable in apple music of the country you haveentered'

############## Bugs ############

if Mode == "Bugs":
  myurl = url
  soup = BeautifulSoup(urlopen(myurl), "lxml")
  kvt = soup.title.get_text()
  name = kvt.replace(' - 벅스', '')
  resolution = Picture_Resoultion # change resolution in form
  artwork_base_url = f'https://image.bugsm.co.kr/album/images/{resolution}'

  album_url = url
  album_id = album_url.split('/')[-1].split('?')[0]
  id = album_id[0:-2]

  artwork_url = f'{artwork_base_url}/{id}/{album_id}.jpg'
  dl = artwork_url
  path = output_path

  wget.download(dl, f'{path}/{name}-cover.jpg')
  print("Downloaded " + name + " Album ArtWork Sucessfully..!!")

In [ ]:

import os,re

#@title ##Upload to your image to [transfer.sh](https://transfer.sh)
#@markdown In *file_name* add your custom name and default extension

file_path = "" #@param {type:"string"}
file_name = "Album_Cover(TheKVT).jpg" #@param {type:"string"}
max_downloads = "4" #@param {type:"string"}
store_days = "1" #@param ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14"]

os.environ['inputFile'] = file_path 
os.environ['maxDl'] = max_downloads
os.environ['daysStore'] = store_days
os.environ['fileName'] = file_name

mylink = !curl -H "Max-Downloads: "$maxDl"" -H "Max-Days: "$daysStore"" --upload-file "$inputFile" https://transfer.sh/"$fileName"
print('Here your image link:-  ' + str(mylink))